In [1]:
import pandas as pd
import numpy as np

In [2]:
# 階層化されたインデックスの操作について考える
# ネストしたインデックスを持つSeries
data = pd.Series(np.random.randn(10), index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1   -0.941457
   2    1.189396
   3    1.601655
b  1   -1.945266
   2    2.000185
   3    0.968771
c  1   -0.481251
   2    0.299320
d  2    0.743704
   3   -0.521326
dtype: float64

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [4]:
# 外側の階層のインデックスを呼ぶことで部分集合を簡潔に呼び出せる
data['b']

1   -1.945266
2    2.000185
3    0.968771
dtype: float64

In [5]:
# 外側のインデックスをスライスする
data['b':'c']

b  1   -1.945266
   2    2.000185
   3    0.968771
c  1   -0.481251
   2    0.299320
dtype: float64

In [6]:
# 外側のインデックスをlocで抽出する
data.loc[['b', 'd']]

b  1   -1.945266
   2    2.000185
   3    0.968771
d  2    0.743704
   3   -0.521326
dtype: float64

In [7]:
# 外側のインデックスを指定した後に、内側のインデックスを指定する
data[:, 2]

a    1.189396
b    2.000185
c    0.299320
d    0.743704
dtype: float64

In [8]:
# ネストしたインデックスはunstackするとDataFrameになる
data.unstack()

,1,2,3
a,-0.941457,1.189396,1.601655
b,-1.945266,2.000185,0.968771
c,-0.481251,0.299320,NaN
d,NaN,0.743704,-0.521326


In [9]:
# stackすることでSeriesに戻せる
data.unstack().stack()

a  1   -0.941457
   2    1.189396
   3    1.601655
b  1   -1.945266
   2    2.000185
   3    0.968771
c  1   -0.481251
   2    0.299320
d  2    0.743704
   3   -0.521326
dtype: float64

In [10]:
# データフレームでは、どちらの軸も階層型インデックスを持てる
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
# 各階層のインデックスに名前をつけることができる
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
# columnの外側のインデックスを指定する
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
# swaplevel: ２つの階層の数値か名前を指定することで、インデックス階層を入れ替えた新しいオブジェクトを返す
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [14]:
# 軸を指定してインデックスをソートできる
frame.sort_index(1)

state     Colorado  Ohio    
color        Green Green Red
key1 key2                   
a    1           2     0   1
     2           5     3   4
b    1           8     6   7
     2          11     9  10

In [15]:
frame.swaplevel(0, 1).sort_index(0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [16]:
# levelを指定することでとりたい要約統計量の階層を指定できる
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [17]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [18]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [19]:
# set_indexdeで、DataFrameの列をインデックスにする
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [20]:
# dropをFalsenにすることでインデックスにした列を残すことができる
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [21]:
# reset_indexで階層型インデックスを列にする/戻すことができる
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
